# Dynamic Recurrent Neural Network

Tensorflow 2.0 implementation of a Reccurent Neural Network (LSTM) that performs computation over sequences with variable lenght. This example is using a toy dataset to classify linear sequences. The genrated sequences have varibale length.

In [1]:
! pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers


In [3]:
# Dataset parameters.
num_classes = 2 # linear sequence or not.
seq_max_len = 20 # Maximum sequence length.
seq_min_len = 5 # Minimum sequence length (before padding).
masking_val = -1 # -1 will represents the mask and be used to pad sequences to a common max length.
max_value = 10000 # Maximum int value.

# Training Parameters
learning_rate = 0.001
training_steps = 2000
batch_size = 64
display_step = 100

# Network Parameters
num_units = 32 # number of neurons for the LSTM layer.

In [4]:
# ====================
#  TOY DATA GENERATOR
# ====================

def toy_sequence_data():
    """ Generate sequence of data with dynamic length.
    This function generates toy samples for training:
    - Class 0: linear sequences (i.e. [1, 2, 3, 4, ...])
    - Class 1: random sequences (i.e. [9, 3, 10, 7,...])

    NOTICE:
    We have to pad each sequence to reach 'seq_max_len' for TensorFlow
    consistency (we cannot feed a numpy array with inconsistent
    dimensions). The dynamic calculation will then be perform and ignore
    the masked value (here -1).
    """
    while True:
        # Set variable sequence length.
        seq_len = random.randint(seq_min_len, seq_max_len)
        rand_start = random.randint(0, max_value - seq_len)
        # Add a random or linear int sequence (50% prob).
        if random.random() < .5:
            # Generate a linear sequence.
            seq = np.arange(start=rand_start, stop=rand_start+seq_len)
            # Rescale values to [0., 1.].
            seq = seq / max_value
            # Pad sequence until the maximum length for dimension consistency.
            # Masking value: -1.
            seq = np.pad(seq, mode='constant', pad_width=(0, seq_max_len-seq_len), constant_values=masking_val)
            label = 0
        else:
            # Generate a random sequence.
            seq = np.random.randint(max_value, size=seq_len)
            # Rescale values to [0., 1.].
            seq = seq / max_value
            # Pad sequence until the maximum length for dimension consistency.
            # Masking value: -1.
            seq = np.pad(seq, mode='constant', pad_width=(0, seq_max_len-seq_len), constant_values=masking_val)
            label = 1
        yield np.array(seq, dtype=np.float32), np.array(label, dtype=np.float32)

In [5]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_generator(toy_sequence_data, output_types=(tf.float32, tf.float32))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [12]:
# Create LSTM Model
class LSTM(Model):
  # Set layers
  def __init__(self):
    super(LSTM, self).__init__()
    # Define a Masking layer with -1 as mask
    self.masking = layers.Masking(mask_value=masking_val)
    # Define a LSTM layer to be applied over the Masking layer
    # Dynamic computation will automatically be performed to ignore -1 values
    self.lstm = layers.LSTM(units=num_units)
    # Output fully connected layer (2 classes: linear or random seq)
    self.out = layers.Dense(num_classes)
  
  # Set forward pass
  def call(self, x, is_training: bool = False):
    # A RNN layer expects a 3-dim input (batch_size, seq_len, num_features)
    x = tf.reshape(x, shape=[-1, seq_max_len, 1])
    # Apply Masking layer
    x = self.masking(x)
    # Apply LSTM layer
    x = self.lstm(x)
    # Apply output layer
    x = self.out(x)
    if not is_training:
      # tf cross entropy expect logits without softmax, so only
      # apply softmax when not training
      x = tf.nn.sotfmax(x)
    return x

# Build LSTM Model
lstm_net = LSTM()

In [8]:
# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Adam optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [9]:
# Optimization process
def run_optimization(x, y):
  # Wrap computation inside a GradientTape for automatic differentiation
  with tf.GradientTape() as g:
    # Forward pass
    pred = lstm_net(x, is_training=True)
    # Compute loss
    loss = cross_entropy_loss(pred, y)

  # Variable to update, i.e. trainable variables
  trainable_variables = lstm_net.trainable_variables

  # Compute gradients
  gradients = g.gradient(loss, trainable_variables)

  # Update weights follwoing gradients
  optimizer.apply_gradients(zip(gradients, trainable_variables))

In [13]:
# Run training for the given number of steps
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
  # Run he optimization to update W and b values
  run_optimization(batch_x, batch_y)

  if step % display_step == 0 or step == 1:
    pred = lstm_net(batch_x, is_training=True)
    loss = cross_entropy_loss(pred, batch_y)
    acc = accuracy(pred, batch_y)

    print(f"step: {step}, loss: {loss}, accuracy: {acc}")

step: 1, loss: 0.6908915042877197, accuracy: 0.53125
step: 100, loss: 0.6361424922943115, accuracy: 0.671875
step: 200, loss: 0.48440638184547424, accuracy: 0.75
step: 300, loss: 0.4346997141838074, accuracy: 0.84375
step: 400, loss: 0.4939959645271301, accuracy: 0.734375
step: 500, loss: 0.47112154960632324, accuracy: 0.75
step: 600, loss: 0.3667265474796295, accuracy: 0.859375
step: 700, loss: 0.4305405616760254, accuracy: 0.71875
step: 800, loss: 0.24336791038513184, accuracy: 0.90625
step: 900, loss: 0.18624168634414673, accuracy: 0.96875
step: 1000, loss: 0.14674553275108337, accuracy: 0.984375
step: 1100, loss: 0.13943040370941162, accuracy: 0.953125
step: 1200, loss: 0.09680555760860443, accuracy: 0.984375
step: 1300, loss: 0.5483678579330444, accuracy: 0.875
step: 1400, loss: 0.33413007855415344, accuracy: 0.875
step: 1500, loss: 0.15386538207530975, accuracy: 0.921875
step: 1600, loss: 0.22036492824554443, accuracy: 0.921875
step: 1700, loss: 0.043732672929763794, accuracy: 1.

In [ ]:
# test_accuracy = lstm_net()